In [ ]:
import pandas as pd
from pathlib import Path
import datetime

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径和常量"""
    # --- 【请修改】输入和输出路径 ---
    INPUT_FILE = Path(r'E:\A智网\业扩分析\12月分析\业扩跟踪用户电量7.1-12.17.xlsx')
    OUTPUT_FILE = INPUT_FILE.parent / '业扩用户电量分析报告.xlsx'

    # --- 源数据列名 ---
    DATE_COL = 'ds'
    CUST_NO_COL = 'cust_no'
    CUST_NAME_COL = 'cust_name'
    LOAD_COL = 'total_eq'
    
    # --- 业务逻辑参数 ---
    # 11月第一周的周一日期 (2025年11月3日是周一)
    FILTER_START_WEEK_MONDAY = pd.Timestamp('2025-11-03')
    
    # --- 输出Sheet名称 ---
    OUTPUT_SHEET_MONTHLY = '月电量'
    OUTPUT_SHEET_WEEKLY_AVG = '周日均电量'

# ==============================================================================
# 2. 数据处理与计算层 (Processing & Calculation Layer)
# ==============================================================================

def preprocess_data(file_path: Path) -> pd.DataFrame:
    """读取并预处理源数据"""
    print(f"正在读取和预处理文件: {file_path.name}")
    if not file_path.exists():
        raise FileNotFoundError(f"输入文件不存在: {file_path}")
        
    df = pd.read_excel(file_path)
    
    # 1. 转换日期列
    df[Config.DATE_COL] = pd.to_datetime(df[Config.DATE_COL].astype(str), format='%Y%m%d')
    
    # 2. 确保电量列是数值类型
    df[Config.LOAD_COL] = pd.to_numeric(df[Config.LOAD_COL], errors='coerce').fillna(0)
    
    # 3. 清理文本
    df[Config.CUST_NAME_COL] = df[Config.CUST_NAME_COL].astype(str).str.strip()
    df[Config.CUST_NO_COL] = df[Config.CUST_NO_COL].astype(str).str.strip()
    
    return df

def generate_monthly_report(df: pd.DataFrame) -> pd.DataFrame:
    """生成月电量汇总报告 (修复Windows下月份格式问题)"""
    print("正在生成【月电量】报告...")
    
    df_monthly = df.copy()
    
    # 1. 提取月份 (使用 dt.month 获取整数，避免 %-m 的兼容性问题)
    #    同时保留 年-月 格式用于排序
    df_monthly['month_int'] = df_monthly[Config.DATE_COL].dt.month
    df_monthly['sort_key'] = df_monthly[Config.DATE_COL].dt.strftime('%Y%m')
    
    # 2. 分组求和
    monthly_sum = df_monthly.groupby([Config.CUST_NO_COL, Config.CUST_NAME_COL, 'month_int', 'sort_key'])[Config.LOAD_COL].sum().reset_index()
    
    # 3. 创建显示的列名 "7月", "8月"
    monthly_sum['月份显示'] = monthly_sum['month_int'].astype(str) + '月'
    
    # 4. 透视
    monthly_report = monthly_sum.pivot_table(
        index=[Config.CUST_NAME_COL, Config.CUST_NO_COL],
        columns='月份显示',
        values=Config.LOAD_COL,
        aggfunc='sum'
    ).fillna(0)
    
    # 5. 重新排序列名 (按 7月, 8月... 顺序，而不是字符串顺序 10月, 11月, 7月)
    #    获取所有存在的月份列
    existing_cols = monthly_report.columns.tolist()
    #    定义标准顺序
    sorted_cols = sorted(existing_cols, key=lambda x: int(x.replace('月', '')))
    
    monthly_report = monthly_report[sorted_cols].reset_index()
    monthly_report.columns.name = None
    monthly_report.rename(columns={Config.CUST_NAME_COL: '用户名称', Config.CUST_NO_COL: '编号'}, inplace=True)
    
    print("【月电量】报告生成完毕。")
    return monthly_report

def generate_weekly_avg_report(df: pd.DataFrame) -> pd.DataFrame:
    """生成周日均电量报告 (修复周范围判定和天数计算)"""
    print("正在生成【周日均电量】报告...")
    
    df_weekly = df.copy()
    
    # 1. 获取全局最大日期 (用于处理最后一周的截止)
    global_max_date = df_weekly[Config.DATE_COL].max()
    print(f"  -> 数据截止日期: {global_max_date.strftime('%Y-%m-%d')}")

    # 2. 计算每一行对应的“周一”日期 (标准周)
    #    weekday: 周一=0, 周日=6
    df_weekly['week_monday'] = df_weekly[Config.DATE_COL] - pd.to_timedelta(df_weekly[Config.DATE_COL].dt.weekday, unit='D')
    
    # 3. 【核心过滤】只保留 11月第一周 (11.3) 及之后的数据
    df_weekly = df_weekly[df_weekly['week_monday'] >= Config.FILTER_START_WEEK_MONDAY].copy()
    
    if df_weekly.empty:
        print("  [警告] 过滤后没有数据！请检查日期范围。")
        return pd.DataFrame()

    # 4. 计算每一周的“标准结束日期” (周日) 和 “实际计费天数”
    #    注意：这里我们创建一个辅助DataFrame来处理周的元数据，确保所有用户在同一周的天数是一样的
    unique_weeks = df_weekly[['week_monday']].drop_duplicates()
    unique_weeks['week_sunday_theoretical'] = unique_weeks['week_monday'] + pd.Timedelta(days=6)
    
    #    实际结束日期 = min(理论周日, 全局最大日期)
    #    例如最后一周：min(12.21, 12.17) = 12.17
    unique_weeks['week_end_real'] = unique_weeks['week_sunday_theoretical'].apply(lambda x: min(x, global_max_date))
    
    #    计算该周的实际天数 (用于求平均)
    unique_weeks['days_count'] = (unique_weeks['week_end_real'] - unique_weeks['week_monday']).dt.days + 1
    
    #    生成列名标签 "11.03-11.09"
    unique_weeks['col_label'] = (
        unique_weeks['week_monday'].dt.strftime('%m.%d') + '-' + 
        unique_weeks['week_end_real'].dt.strftime('%m.%d')
    )
    
    # 5. 将周元数据合并回主数据
    df_weekly = pd.merge(df_weekly, unique_weeks, on='week_monday', how='left')
    
    # 6. 分组求和 (按用户和周标签)
    weekly_sum = df_weekly.groupby([Config.CUST_NO_COL, Config.CUST_NAME_COL, 'col_label', 'week_monday', 'days_count'])[Config.LOAD_COL].sum().reset_index()
    
    # 7. 【核心计算】计算日均 = 总电量 / 该周实际天数
    weekly_sum['daily_avg'] = weekly_sum[Config.LOAD_COL] / weekly_sum['days_count']
    
    # 8. 透视
    weekly_report = weekly_sum.pivot_table(
        index=[Config.CUST_NAME_COL, Config.CUST_NO_COL],
        columns='col_label',
        values='daily_avg',
        aggfunc='sum' # 这里sum其实就是取唯一值，因为前面已经group过了
    ).fillna(0)
    
    # 9. 对列名进行排序 (确保时间顺序)
    #    我们需要根据 col_label 对应的 week_monday 来排序
    #    创建一个 标签->周一 的映射字典
    label_order_map = unique_weeks.set_index('col_label')['week_monday'].to_dict()
    #    根据周一日期对列名进行排序
    sorted_columns = sorted(weekly_report.columns, key=lambda x: label_order_map.get(x, pd.Timestamp.min))
    
    weekly_report = weekly_report[sorted_columns].reset_index()
    weekly_report.columns.name = None
    weekly_report.rename(columns={Config.CUST_NAME_COL: '用户名称', Config.CUST_NO_COL: '编号'}, inplace=True)
    
    print("【周日均电量】报告生成完毕。")
    return weekly_report

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print(f"--- 开始处理业扩跟踪用户电量数据 ---")
    try:
        # 1. 读取和预处理数据
        df = preprocess_data(Config.INPUT_FILE)
        
        # 2. 生成月电量报告
        monthly_report = generate_monthly_report(df)
        
        # 3. 生成周日均电量报告
        weekly_avg_report = generate_weekly_avg_report(df)
        
        # 4. 将两个报告写入到同一个Excel文件的不同Sheet中
        print(f"\n正在将结果写入到文件: {Config.OUTPUT_FILE}")
        with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
            monthly_report.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_MONTHLY, index=False)
            print(f"  -> 已写入Sheet: '{Config.OUTPUT_SHEET_MONTHLY}'")
            
            if not weekly_avg_report.empty:
                weekly_avg_report.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_WEEKLY_AVG, index=False)
                print(f"  -> 已写入Sheet: '{Config.OUTPUT_SHEET_WEEKLY_AVG}'")
            else:
                print(f"  -> [警告] 周报表为空，未写入。")
            
        print(f"\n--- 全部任务成功完成！ ---")

    except Exception as e:
        print(f"[致命错误] 执行主流程时发生错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import font_manager
import matplotlib.ticker as ticker
import os

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    # --- 输入文件路径 ---
    INPUT_FILE = Path(r'E:\A智网\业扩分析\12月分析\业扩用户电量分析报告.xlsx')
    
    # --- 输出文件夹路径 ---
    OUTPUT_DIR = INPUT_FILE.parent / '用户电量图表'

    # --- 【关键】请在这里填入您想要绘图的用户名称列表 ---
    TARGET_USER_NAMES = [
        "东风商用车有限公司",
        "国家管网集团联合管道有限责任公司西气东输分公司武汉管理处",
        "大冶市华兴玻璃有限公司",
        "广水新煌循环资源有限公司",
        "应城市新都化工有限责任公司",
        "武汉钢铁有限公司",
        "武汉高科国有控股集团有限公司",
        "湖北宜化新能源有限公司",
        "湖北徽阳新材料有限公司",
        "湖北新祥云新材料有限公司",
        "湖北日盛科技有限公司",
        "湖北洪伯车辆有限公司",
        "湖北瑞达智造装备有限公司",
        "湖北金茂环保科技有限公司",
        "维达力科技股份有限公司",
        "荆门源晗电池材料有限公司",
        "长江沿岸铁路集团湖北有限公司",
        "黄石新兴管业有限公司"
        # ... 添加更多用户
    ]

    # --- Sheet名称 ---
    SHEET_MONTHLY = '月电量'
    SHEET_WEEKLY = '周日均电量'

    # --- 字体配置 ---
    TARGET_FONT = 'SimHei'  # 黑体
    
    # --- 绘图样式 ---
    FIG_SIZE = (20, 8)      # 画布大小
    LINE_WIDTH = 3.0        # 线宽
    MARKER_SIZE = 9         # 标记大小
    COLOR_MONTHLY = '#1f77b4' # 蓝色
    COLOR_WEEKLY = '#ff7f0e'  # 橙色

# ==============================================================================
# 2. 辅助函数
# ==============================================================================
def get_font_properties(font_name):
    """获取中文字体属性"""
    font_files = font_manager.fontManager.ttflist
    for f in font_files:
        if f.name == font_name:
            return font_manager.FontProperties(fname=f.fname)
    return font_manager.FontProperties(family='sans-serif')

def plot_single_user(df_monthly, df_weekly, user_name):
    """
    为单个用户绘制图表并保存
    """
    print(f"正在处理用户: {user_name} ...")

    # 1. 数据准备
    user_monthly = df_monthly[df_monthly['用户名称'] == user_name]
    user_weekly = df_weekly[df_weekly['用户名称'] == user_name]

    if user_monthly.empty and user_weekly.empty:
        print(f"  [跳过] 未在任何表中找到用户: {user_name}")
        return

    # 2. 准备字体
    font_prop = get_font_properties(Config.TARGET_FONT)
    font_title = font_prop.copy()
    font_title.set_size(18)
    font_title.set_weight('bold')
    font_label = font_prop.copy()
    font_label.set_size(12)
    font_tick = font_prop.copy()
    font_tick.set_size(10)

    # 3. 创建画布
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=Config.FIG_SIZE)
    
    fig.suptitle(f"用户电量趋势分析报告 - {user_name}", fontproperties=font_title, y=0.93)

    # --- 左图：月度趋势 ---
    if not user_monthly.empty:
        month_cols = [c for c in df_monthly.columns if c not in ['用户名称', '编号']]
        y_values = user_monthly.iloc[0][month_cols].values
        
        ax1.plot(month_cols, y_values, marker='o', linewidth=Config.LINE_WIDTH, 
                 markersize=Config.MARKER_SIZE, color=Config.COLOR_MONTHLY)
        
        ax1.set_title("月度总电量趋势 (2025年)", fontproperties=font_label, pad=10)
        ax1.set_ylabel("电量 (千瓦时)", fontproperties=font_label)
        ax1.grid(True, linestyle='--', alpha=0.6)
        
        # 【核心修改】设置Y轴范围：从0开始，上限为最大值的1.2倍
        # 这样可以让波动看起来更平缓，更真实
        if len(y_values) > 0:
            max_val = y_values.max()
            ax1.set_ylim(bottom=0, top=max_val * 1.25)

        # 数据标签
        for x, y in zip(month_cols, y_values):
            ax1.text(x, y, f'{y:,.0f}', ha='center', va='bottom', fontsize=9)
            
        ax1.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
        for label in ax1.get_xticklabels() + ax1.get_yticklabels():
            label.set_fontproperties(font_tick)
    else:
        ax1.text(0.5, 0.5, '暂无月度数据', ha='center', va='center', fontproperties=font_title)

    # --- 右图：周度趋势 ---
    if not user_weekly.empty:
        week_cols = [c for c in df_weekly.columns if c not in ['用户名称', '编号']]
        y_values = user_weekly.iloc[0][week_cols].values
        
        ax2.plot(week_cols, y_values, marker='s', linewidth=Config.LINE_WIDTH, 
                 markersize=Config.MARKER_SIZE, color=Config.COLOR_WEEKLY, linestyle='--')
        
        ax2.set_title("周度日均电量趋势 (11月起)", fontproperties=font_label, pad=10)
        ax2.set_ylabel("日均电量 (千瓦时/天)", fontproperties=font_label)
        ax2.grid(True, linestyle='--', alpha=0.6)
        
        # 【核心修改】设置Y轴范围：从0开始
        if len(y_values) > 0:
            max_val = y_values.max()
            ax2.set_ylim(bottom=0, top=max_val * 1.25)
        
        ax2.set_xticklabels(week_cols, rotation=45, ha='right')
        
        for x, y in zip(week_cols, y_values):
            ax2.text(x, y, f'{y:,.0f}', ha='center', va='bottom', fontsize=9)

        ax2.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
        for label in ax2.get_xticklabels() + ax2.get_yticklabels():
            label.set_fontproperties(font_tick)
    else:
        ax2.text(0.5, 0.5, '暂无周度数据', ha='center', va='center', fontproperties=font_title)

    # 4. 保存图片
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    safe_name = "".join([c for c in user_name if c.isalpha() or c.isdigit() or c in " _-()（）"]).strip()
    output_path = Config.OUTPUT_DIR / f"{safe_name}_电量趋势.png"
    
    try:
        plt.savefig(output_path, dpi=150)
        print(f"  -> 已保存: {output_path.name}")
    except Exception as e:
        print(f"  [错误] 保存失败: {e}")
    
    plt.close(fig)

# ==============================================================================
# 3. 主流程
# ==============================================================================
def main():
    if not Config.INPUT_FILE.exists():
        print(f"[致命错误] 输入文件不存在: {Config.INPUT_FILE}")
        return

    if not Config.TARGET_USER_NAMES:
        print("[提示] 请在 Config.TARGET_USER_NAMES 中填入您想查看的用户名称。")
        return

    if not Config.OUTPUT_DIR.exists():
        Config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        print(f"已创建输出文件夹: {Config.OUTPUT_DIR}")

    try:
        print("正在读取数据...")
        df_monthly = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_MONTHLY)
        df_weekly = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_WEEKLY)
        
        df_monthly['用户名称'] = df_monthly['用户名称'].astype(str).str.strip()
        df_weekly['用户名称'] = df_weekly['用户名称'].astype(str).str.strip()

        print(f"开始批量生成 {len(Config.TARGET_USER_NAMES)} 个用户的图表...")
        
        for user in Config.TARGET_USER_NAMES:
            plot_single_user(df_monthly, df_weekly, user)
            
        print(f"\n--- 全部完成！所有图片已保存在: {Config.OUTPUT_DIR} ---")

    except Exception as e:
        print(f"[错误] 执行过程中发生错误: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

正在读取数据...
开始批量生成 18 个用户的图表...
正在处理用户: 东风商用车有限公司 ...


C:\Users\Joe-h\AppData\Local\Temp\ipykernel_32528\3827925562.py:140: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(week_cols, rotation=45, ha='right')


  -> 已保存: 东风商用车有限公司_电量趋势.png
正在处理用户: 国家管网集团联合管道有限责任公司西气东输分公司武汉管理处 ...


C:\Users\Joe-h\AppData\Local\Temp\ipykernel_32528\3827925562.py:140: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(week_cols, rotation=45, ha='right')


  -> 已保存: 国家管网集团联合管道有限责任公司西气东输分公司武汉管理处_电量趋势.png
正在处理用户: 大冶市华兴玻璃有限公司 ...


...

  -> 已保存: 荆门源晗电池材料有限公司_电量趋势.png
正在处理用户: 长江沿岸铁路集团湖北有限公司 ...


C:\Users\Joe-h\AppData\Local\Temp\ipykernel_32528\3827925562.py:140: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(week_cols, rotation=45, ha='right')


  -> 已保存: 长江沿岸铁路集团湖北有限公司_电量趋势.png
正在处理用户: 黄石新兴管业有限公司 ...


C:\Users\Joe-h\AppData\Local\Temp\ipykernel_32528\3827925562.py:140: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(week_cols, rotation=45, ha='right')


  -> 已保存: 黄石新兴管业有限公司_电量趋势.png

--- 全部完成！所有图片已保存在: E:\A智网\业扩分析\12月分析\用户电量图表 ---
